In [1]:
folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper/'.format(folder)
bed_folder = '{}/bed'.format(folder)



splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

In [3]:
import pandas as pd
import six

In [5]:
constitutive_junctions = pd.read_csv('{}/constitutive_junctions.csv'.format(constitutive_feature_folder))
six.print_(constitutive_junctions.shape)
constitutive_junctions.head()

(25004, 3)


,exon,direction,junction
0,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342288:-
1,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1341605:-
2,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342215:-
3,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342367:-
4,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342510:-


In [7]:
from outrigger.region import Region

In [8]:
constitutive_junctions['exon_region'] = constitutive_junctions['exon'].map(Region)
constitutive_junctions['junction_region'] = constitutive_junctions['junction'].map(Region)
constitutive_junctions.head()

,exon,direction,junction,exon_region,junction_region
0,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342288:-,exon:chr1:1341189-1341266:-,chr1:1341267-1342288:-
1,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1341605:-,exon:chr1:1341189-1341266:-,chr1:1341267-1341605:-
2,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342215:-,exon:chr1:1341189-1341266:-,chr1:1341267-1342215:-
3,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342367:-,exon:chr1:1341189-1341266:-,chr1:1341267-1342367:-
4,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342510:-,exon:chr1:1341189-1341266:-,chr1:1341267-1342510:-


In [10]:
constitutive_junctions['exon_length'] = constitutive_junctions['exon_region'].map(len)
constitutive_junctions['junction_length'] = constitutive_junctions['junction_region'].map(len)
constitutive_junctions.head()

,exon,direction,junction,exon_region,junction_region,exon_length,junction_length
0,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342288:-,exon:chr1:1341189-1341266:-,chr1:1341267-1342288:-,78,1022
1,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1341605:-,exon:chr1:1341189-1341266:-,chr1:1341267-1341605:-,78,339
2,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342215:-,exon:chr1:1341189-1341266:-,chr1:1341267-1342215:-,78,949
3,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342367:-,exon:chr1:1341189-1341266:-,chr1:1341267-1342367:-,78,1101
4,exon:chr1:1341189-1341266:-,downstream,chr1:1341267-1342510:-,exon:chr1:1341189-1341266:-,chr1:1341267-1342510:-,78,1244


In [16]:
exon_lengths = constitutive_junctions.set_index('exon')['exon_length']
exon_lengths= exon_lengths[~exon_lengths.index.duplicated()]
exon_lengths.head()

exon
exon:chr1:1341189-1341266:-      78
exon:chr1:1341189-1342399:-    1211
exon:chr1:1653035-1653150:-     116
exon:chr1:1650767-1650894:-     128
exon:chr1:1650797-1650894:-      98
Name: exon_length, dtype: int64

In [23]:
exon_lengths.to_csv('{}/exon_length.csv'.format(constitutive_feature_folder))

## Intron lengths: Sum of junction and exon lengths

In [18]:
mean_junction_lengths = constitutive_junctions.groupby(['exon', 'direction'])['junction_length'].mean()
mean_junction_lengths.head()

exon                              direction 
exon:chr10:102039881-102039985:-  downstream     705.0
                                  upstream      4631.5
exon:chr10:102040691-102040810:-  downstream    3575.5
                                  upstream       550.0
exon:chr10:102045854-102045970:-  downstream     255.0
Name: junction_length, dtype: float64

In [19]:
flanking_intron_lengths = mean_junction_lengths.groupby(level=0).sum()
flanking_intron_lengths.head()

exon
exon:chr10:102039881-102039985:-     5336.500000
exon:chr10:102040691-102040810:-     4125.500000
exon:chr10:102045854-102045970:-    10974.666667
exon:chr10:102045854-102046089:-    10991.666667
exon:chr10:102286732-102286831:-     4030.500000
Name: junction_length, dtype: float64

In [20]:
intron_lengths = flanking_intron_lengths + exon_lengths
intron_lengths.head()

exon
exon:chr10:102039881-102039985:-     5441.500000
exon:chr10:102040691-102040810:-     4245.500000
exon:chr10:102045854-102045970:-    11091.666667
exon:chr10:102045854-102046089:-    11227.666667
exon:chr10:102286732-102286831:-     4130.500000
dtype: float64

In [22]:
intron_lengths.to_csv('{}/intron_length.csv'.format(constitutive_feature_folder))